# Distributional Semantic Models in Python
EMLAR, 2021

TODO write some intro to this

In [2]:
#First we import some libraries that will be useful
import nltk #Natural Language Toolkit
import numpy as np
import pandas as pd # For data frames 
from sklearn.metrics.pairwise import cosine_similarity 
from collections
import pattern
from pattern.en import lemma
#nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

## Data Preprocessing

Change data? 
e.g.
https://www.gutenberg.org/ebooks/bookshelves/search/?query=children|christmas|child|school



In [ ]:
#Load data from book that we have previously downloaded from Gutenberg project
raw = open("pg6328.txt", 'r').read()

#Let's have a look at the data
print(raw)

In [ ]:
#The data is a complete text. We need it separated by sentences and words.
#This process is called "tokenization".
#First we need to download a tokenizer from NLTK: ##maybe change to spacy and do everything with spacy?
#nltk.download('punkt')

#Now let's tokenize each sentence.
#nltk.sent_tokenize gives us a method to do so
sentences = nltk.sent_tokenize(raw)

#And we tokenize all the words in each sentence and collect them together 
tokenized = []
header=True
for sentence in sentences:
    if header and sentence.startswith("CONTENTS"):
        header = False
    if not header: #We ignore everything before the table of contents
        tokenized.append(nltk.wordpunct_tokenize(sentence))
    

#Let's look at the words. Do you spot any problem? (need to lemmatize and lowercase)
print(tokenized)

In [ ]:
# Lowercase and lemmatize
# TO DO!!! (nltk or spacy)


In [ ]:
#Let's inspect our data (here we could do a bunch of stuff, not sure if necessary)

#We first build a dictionary with word types and their frequencies
word_frequencies = {}
for sentence in tokenized:
    for word in sentence:
        word_frequencies[word] = word_frequencies.get(word,0) + 1


#We first look at some of the most frequent words.
# We construct a list of words ordered from most frequent to most infrequent
sorted_keys = sorted(word_frequencies, key=word_frequencies.get, reverse=True)
print("These are the 10 most frequent words: ", sorted_keys[:10])
#todo decide whether to remove the stop words?


#Not sure this is interesting:
#Let's look at the singletons (i.e. words with frequency 1)
singletons=[]
for word, frequency in word_frequencies.items():
    if frequency == 1:
        singletons.append(word)
print("We have {} singletons.".format(len(singletons)))

#TO DO Look at frequency distribution?

In [ ]:
# It is generally better to restrict models to words with a minimum frequency.
# We define a minimum frequency threshold of 10 and filter the words:
minfreq=10
target_freqs = dict([(word,freq) for word,freq in word_frequencies.items() if freq>minfreq])
#Here we have a list of all the words (targets) for our model:
targets = target_freqs.keys()
vocabulary_size=len(target_freqs)
print(vocabulary_size)

In [ ]:
# We associate a numerical index to each word, which we later use to locate the word in the co-occurrence matrix
w2i = {w: i for i, w in enumerate(targets)}
i2w = {i: w for i, w in enumerate(targets)}

#Example:
print("The code for the word \"cave\" is {}".format(w2i["cave"]))

In [ ]:
# This function is used to build the co-occurrence matrix
def calculate_cooccurrences(tokenized, vocabulary_size, window):
    matrix = np.zeros([vocabulary_size,vocabulary_size]) 
    for sentence in tokenized:
        for position,word in enumerate(sentence):                
            for j in range(max(position-window,0),min(position+window+1,len(sentence))):
                context=sentence[j]
                if j!=position and word in targets and context in targets: 
                    matrix[w2i[word]][w2i[context]]+=1
    return matrix


#This function will give us the co-occurrence between two words
def get_cooccurrence(word1, word2):
    return co_occurrence_counts[w2i[word1]][w2i[word2]]

#We now compute the co-occurrences in our tokenized text
co_occurrence_counts=calculate_cooccurrences(tokenized, vocabulary_size, 2)

#Let's have a look at some co-occurrences:
print(get_cooccurrence("travelled", "distance"))
print(get_cooccurrence("next", "morning"))

In [ ]:
## to do: if we apply (P)PMI, SVD, etc this would be the place

In [ ]:
## Similarity

In [ ]:
similarities = cosine_similarity(co_occurrence_counts)
print(similarities)

#This function will give us the co-occurrence between two words
def get_similarities(word1, word2):
    return similarities[w2i[word1]][w2i[word2]]


In [ ]:
print(get_similarities("eat", "drink"))
print(get_similarities("lamp","door"))
print(get_similarities("lamp","drink"))

## From here we can do any tipe of analyses 
#(e.g. correlate with similarity judgements, find closest neighbours to one word, compare similarities between two different books, or sth like that)

# CHILDES DATA

In [3]:
thomas = pd.read_csv('thomas.csv')
thomas.head(n = 10)

,utterance
0,thomas whats the matter whats the matter
1,would you like some more juice
2,oh that was a nice squeal thomas
3,wasnt it
4,wow juice
5,thomas
6,it has just gone eight oclock on wednesday mor...
7,whose shoe is it
8,its breakfast time
9,what is it


In [41]:
# Now let's tokenize each sentence
tokenized = []
for utterance in thomas['utterance']:
     tokenized.append(nltk.wordpunct_tokenize(utterance))
        
# Add the tokenized list to the data frame
thomas['tokenized'] = tokenized
thomas.head(n = 10)

,utterance,tokenized,lemmatized
0,thomas whats the matter whats the matter,"[thomas, whats, the, matter, whats, the, matter]","[thoma, what, the, matter, what, the, matter]"
1,would you like some more juice,"[would, you, like, some, more, juice]","[would, you, like, some, more, juice]"
2,oh that was a nice squeal thomas,"[oh, that, was, a, nice, squeal, thomas]","[oh, that, be, a, nice, squeal, thoma]"
3,wasnt it,"[wasnt, it]","[wasnt, it]"
4,wow juice,"[wow, juice]","[wow, juice]"
5,thomas,[thomas],[thoma]
6,it has just gone eight oclock on wednesday mor...,"[it, has, just, gone, eight, oclock, on, wedne...","[it, have, just, go, eight, oclock, on, wednes..."
7,whose shoe is it,"[whose, shoe, is, it]","[whose, shoe, be, it]"
8,its breakfast time,"[its, breakfast, time]","[it, breakfast, time]"
9,what is it,"[what, is, it]","[what, be, it]"


In [22]:
lemmatized = []
for utt in range(len(thomas['utterance'])):
    utterance = []
    for word in range(len(thomas.loc[utt, 'tokenized'])):
        utterance.append(lemma(thomas.loc[utt, 'tokenized'][word]))
    lemmatized.append(utterance)
    
# Add the tokenized list to the data frame
thomas['lemmatized'] = lemmatized
thomas.head(n = 10)

,utterance,tokenized,lemmatized
0,thomas whats the matter whats the matter,"[thomas, whats, the, matter, whats, the, matter]","[thoma, what, the, matter, what, the, matter]"
1,would you like some more juice,"[would, you, like, some, more, juice]","[would, you, like, some, more, juice]"
2,oh that was a nice squeal thomas,"[oh, that, was, a, nice, squeal, thomas]","[oh, that, be, a, nice, squeal, thoma]"
3,wasnt it,"[wasnt, it]","[wasnt, it]"
4,wow juice,"[wow, juice]","[wow, juice]"
5,thomas,[thomas],[thoma]
6,it has just gone eight oclock on wednesday mor...,"[it, has, just, gone, eight, oclock, on, wedne...","[it, have, just, go, eight, oclock, on, wednes..."
7,whose shoe is it,"[whose, shoe, is, it]","[whose, shoe, be, it]"
8,its breakfast time,"[its, breakfast, time]","[it, breakfast, time]"
9,what is it,"[what, is, it]","[what, be, it]"


In [30]:
from itertools import chain
lemma_list = list(chain.from_iterable(thomas['lemmatized']))

In [38]:
freq = collections.Counter(lemma_list)
freq.most_common(20)

[('you', 72546),
 ('the', 70985),
 ('be', 57763),
 ('it', 55045),
 ('a', 51713),
 ('and', 46044),
 ('that', 37012),
 ('to', 29431),
 ('have', 26805),
 ('i', 26198),
 ('do', 25902),
 ('we', 22783),
 ('what', 20808),
 ('go', 20189),
 ('on', 19683),
 ('oh', 19481),
 ('in', 19135),
 ('get', 16988),
 ('no', 15717),
 ('there', 15628)]

In [39]:
word_frequencies = {}
for sentence in thomas['lemmatized']:
    for word in sentence:
        word_frequencies[word] = word_frequencies.get(word,0) + 1

In [40]:
sorted_keys = sorted(word_frequencies, key=word_frequencies.get, reverse=True)
print("These are the 10 most frequent words: ", sorted_keys[:10])

These are the 10 most frequent words:  ['you', 'the', 'be', 'it', 'a', 'and', 'that', 'to', 'have', 'i']


In [50]:
# We define a minimum frequency threshold of 10 and filter the words:
minfreq = 10
target_freqs = dict([(word,freq) for word,freq in word_frequencies.items() if freq>minfreq])
#Here we have a list of all the words (targets) for our model:
targets = target_freqs.keys()
vocabulary_size=len(target_freqs)
print(vocabulary_size)
target_freqs

3196


{'thoma': 14307,
 'what': 20808,
 'the': 70985,
 'matter': 664,
 'would': 5144,
 'you': 72546,
 'like': 11822,
 'some': 8215,
 'more': 3168,
 'juice': 1301,
 'oh': 19481,
 'that': 37012,
 'be': 57763,
 'a': 51713,
 'nice': 5210,
 'squeal': 51,
 'wasnt': 1628,
 'it': 55045,
 'wow': 404,
 'have': 26805,
 'just': 9285,
 'go': 20189,
 'eight': 388,
 'oclock': 419,
 'on': 19683,
 'wednesday': 178,
 'morn': 1605,
 'whose': 106,
 'shoe': 747,
 'breakfast': 765,
 'time': 2429,
 'bottle': 294,
 'of': 15165,
 'daddy': 3398,
 'where': 3940,
 'thomas': 330,
 'orange': 1224,
 'alright': 1357,
 'let': 3594,
 'put': 6804,
 'your': 13208,
 'and': 46044,
 'sock': 357,
 'come': 7088,
 'sweetheart': 467,
 'mummy': 6803,
 'pour': 286,
 'drop': 896,
 'water': 1627,
 'in': 19135,
 'to': 29431,
 'down': 4048,
 'noise': 1095,
 'background': 18,
 'kettle': 75,
 'boil': 84,
 'so': 6081,
 'can': 11663,
 'make': 3594,
 'herself': 82,
 'cup': 801,
 'tea': 1360,
 'these': 2652,
 'those': 2357,
 'ill': 2600,
 'get':

In [44]:
targets

dict_keys(['thoma', 'what', 'the', 'matter', 'would', 'you', 'like', 'some', 'more', 'juice', 'oh', 'that', 'be', 'a', 'nice', 'squeal', 'wasnt', 'it', 'wow', 'have', 'just', 'go', 'eight', 'oclock', 'on', 'wednesday', 'morn', 'whose', 'shoe', 'breakfast', 'time', 'bottle', 'of', 'daddy', 'where', 'thomas', 'orange', 'alright', 'let', 'put', 'your', 'and', 'sock', 'come', 'sweetheart', 'mummy', 'pour', 'drop', 'water', 'in', 'to', 'down', 'noise', 'background', 'kettle', 'boil', 'so', 'can', 'make', 'herself', 'cup', 'tea', 'these', 'those', 'ill', 'get', 'do', 'say', 'please', 'need', 'clean', 'because', 'we', 'out', 'thi', 'afternoon', 'thank', 'drink', 'she', 'blue', 'bus', 'ah', 'i', 'see', 'ye', 'over', 'there', 'balloon', 'right', 'how', 'many', 'baby', 'trouser', 'two', 'theyre', 'my', 'count', 'four', 'thomasthetankengine', 'one', 'three', 'who', 'po', 'duck', 'dipsy', 'slipper', 'arent', 'they', 'coach', 'with', 'front', 'call', 'look', 'at', 'hi', 'book', 'now', 'annie', 'cla

In [46]:
# We associate a numerical index to each word, which we later use to locate the word in the co-occurrence matrix
w2i = {w: i for i, w in enumerate(targets)}
i2w = {i: w for i, w in enumerate(targets)}

#Example:
print("The code for the word \"biscuit\" is {}".format(w2i["biscuit"]))

The code for the word "biscuit" is 125


In [48]:
# This function is used to build the co-occurrence matrix
def calculate_cooccurrences(tokenized, vocabulary_size, window):
    matrix = np.zeros([vocabulary_size,vocabulary_size]) 
    for sentence in tokenized:
        for position,word in enumerate(sentence):                
            for j in range(max(position-window,0),min(position+window+1,len(sentence))):
                context=sentence[j]
                if j!=position and word in targets and context in targets: 
                    matrix[w2i[word]][w2i[context]]+=1
    return matrix


#This function will give us the co-occurrence between two words
def get_cooccurrence(word1, word2):
    return co_occurrence_counts[w2i[word1]][w2i[word2]]

#We now compute the co-occurrences in our tokenized text
co_occurrence_counts=calculate_cooccurrences(tokenized, vocabulary_size, 2)

0.0


In [49]:
#Let's have a look at some co-occurrences:
print(get_cooccurrence("glass", "milk"))

2.0
